In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [16]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [17]:
df = fetch_hourly_rides(12)

2025-03-05 11:43:00,550 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 11:43:00,554 INFO: Initializing external client
2025-03-05 11:43:00,554 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 11:43:01,104 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.28s) 


In [18]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 23:00:00+00:00,209,4
1,2025-03-05 00:00:00+00:00,35,0
2,2025-03-05 00:00:00+00:00,38,0
3,2025-03-04 23:00:00+00:00,206,0
4,2025-03-04 23:00:00+00:00,215,2
...,...,...,...
4531,2025-03-05 16:00:00+00:00,115,0
4532,2025-03-05 16:00:00+00:00,224,5
4533,2025-03-05 16:00:00+00:00,227,0
4534,2025-03-05 16:00:00+00:00,19,0


In [19]:
df_pred = fetch_predictions(12)

2025-03-05 11:43:10,021 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 11:43:10,024 INFO: Initializing external client
2025-03-05 11:43:10,025 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 11:43:10,625 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.59s) 


In [20]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,230,215.0,2025-03-05 01:00:00+00:00
1,42,7.0,2025-03-05 01:00:00+00:00
2,99,0.0,2025-03-05 01:00:00+00:00
3,235,0.0,2025-03-05 01:00:00+00:00
4,63,0.0,2025-03-05 01:00:00+00:00
...,...,...,...
4271,179,0.0,2025-03-05 17:00:00+00:00
4272,145,3.0,2025-03-05 17:00:00+00:00
4273,78,0.0,2025-03-05 17:00:00+00:00
4274,252,0.0,2025-03-05 17:00:00+00:00


In [21]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [22]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-05 01:00:00+00:00,261,1,19.0
1,2025-03-05 01:00:00+00:00,252,0,0.0
2,2025-03-05 01:00:00+00:00,249,36,136.0
3,2025-03-05 01:00:00+00:00,237,7,40.0
4,2025-03-05 01:00:00+00:00,50,5,34.0
...,...,...,...,...
4019,2025-03-05 16:00:00+00:00,115,0,0.0
4020,2025-03-05 16:00:00+00:00,224,5,4.0
4021,2025-03-05 16:00:00+00:00,227,0,0.0
4022,2025-03-05 16:00:00+00:00,19,0,0.0


In [23]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [24]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
194,2025-03-05 01:00:00+00:00,2,0,0.0,0.0
461,2025-03-05 02:00:00+00:00,2,0,0.0,0.0
549,2025-03-05 03:00:00+00:00,2,0,0.0,0.0
821,2025-03-05 04:00:00+00:00,2,0,0.0,0.0
1215,2025-03-05 05:00:00+00:00,2,0,0.0,0.0
...,...,...,...,...,...
2792,2025-03-05 12:00:00+00:00,263,100,71.0,-29.0
3107,2025-03-05 13:00:00+00:00,263,108,25.0,-83.0
3448,2025-03-05 14:00:00+00:00,263,136,127.0,-9.0
3738,2025-03-05 15:00:00+00:00,263,153,111.0,-42.0


In [25]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-05 01:00:00+00:00,261,1,19.0,18.0
1,2025-03-05 01:00:00+00:00,252,0,0.0,0.0
2,2025-03-05 01:00:00+00:00,249,36,136.0,100.0
3,2025-03-05 01:00:00+00:00,237,7,40.0,33.0
4,2025-03-05 01:00:00+00:00,50,5,34.0,29.0
...,...,...,...,...,...
4019,2025-03-05 16:00:00+00:00,115,0,0.0,0.0
4020,2025-03-05 16:00:00+00:00,224,5,4.0,-1.0
4021,2025-03-05 16:00:00+00:00,227,0,0.0,0.0
4022,2025-03-05 16:00:00+00:00,19,0,0.0,0.0


In [28]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

mae_by_hour

,pickup_hour,MAE
0,2025-03-05 01:00:00+00:00,13.235060
1,2025-03-05 02:00:00+00:00,13.585657
2,2025-03-05 03:00:00+00:00,14.434263
3,2025-03-05 04:00:00+00:00,16.868526
4,2025-03-05 05:00:00+00:00,17.159363
5,2025-03-05 06:00:00+00:00,9.513944
6,2025-03-05 07:00:00+00:00,11.597610
7,2025-03-05 08:00:00+00:00,11.952191
8,2025-03-05 09:00:00+00:00,9.694444
9,2025-03-05 10:00:00+00:00,9.658730
